# Import libraries

In [17]:
import os
from pathlib import Path
import subprocess
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import mapping
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
from shapely.geometry import box
import rasterio as rio
import earthpy as et
import earthpy.plot as ep
#from osgeo import gdal
# Prettier plotting with seaborn
sns.set(font_scale=1.5)
from datetime import datetime

In [14]:
!pwd

/home/c4ubuntu/projDir/Mauritius_IAS


# Clip Tifs

In [69]:
# List filepaths for all bands in the scence
FileList = glob.glob(os.path.join(r'/home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100','*wc2.1_30s_bioc*.tif'))

# Define clipping extent
xmin, ymin, xmax,ymax = (56.59166666666681, -20.525000000000063, 63.50000000000015, -10.341666666666725) # INSERT HERE THE CORRECT COORDINATES

# Generate gdalwarp command for each band
command = ""

for fp in FileList:
    inputfile = fp
    outputfile = inputfile[:-4] + "_clip.tif"
    
    command += "gdalwarp -te %s %s %s %s %s %s \n" % (xmin, ymin, xmax, ymax, inputfile, outputfile)
    
# Write the commands to an .sh file
cmd_file = "ClipGCM.sh"
f = open(os.path.join(cmd_file), 'w')

f.write(command)
f.close()

In [70]:
!bash ClipGCM.sh

Creating output file that is 829P x 1222L.
Processing /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_GFDL-ESM4_ssp126_2081-2100.tif [1/1] : 0Using internal nodata values (e.g. nan) for image /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_GFDL-ESM4_ssp126_2081-2100.tif.
Copying nodata values from source /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_GFDL-ESM4_ssp126_2081-2100.tif to destination /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_GFDL-ESM4_ssp126_2081-2100_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 829P x 1222L.
Processing /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_UKESM1-0-LL_ssp126_2081-2100.tif [1/1] : 0Using internal nodata values (e.g. nan) for image /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_UKESM1-0-LL_ssp126_2081-2100.tif.

...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 829P x 1222L.
Processing /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_FIO-ESM-2-0_ssp126_2081-2100.tif [1/1] : 0Using internal nodata values (e.g. nan) for image /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_FIO-ESM-2-0_ssp126_2081-2100.tif.
Copying nodata values from source /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_FIO-ESM-2-0_ssp126_2081-2100.tif to destination /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_FIO-ESM-2-0_ssp126_2081-2100_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 829P x 1222L.
Processing /home/Azure_linux_admin/Projdir/Mauritius_IAS/SSP126/2081_2100/wc2.1_30s_bioc_GISS-E2-1-G_ssp126_2081-2100.tif [1/1] : 0Using internal nodata values (e.g. nan) for image /home/Azure_linux_admin/Projdir/Mauritius_I

In [74]:
sorted(os.listdir('SSP126/2061_2080/'))

['ACCESS-CM2',
 'ACCESS-ESM1-5',
 'BCC-CSM2-MR',
 'CMCC-ESM2',
 'CNRM-CM6-1',
 'CNRM-CM6-1-HR',
 'CNRM-ESM2-1',
 'CanESM5',
 'CanESM5-CanOE',
 'EC-Earth3-Veg',
 'EC-Earth3-Veg-LR',
 'FIO-ESM-2-0',
 'GFDL-ESM4',
 'GISS-E2-1-G',
 'GISS-E2-1-H',
 'HadGEM3-GC31-LL',
 'INM-CM4-8',
 'INM-CM5-0',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MIROC6',
 'MPI-ESM1-2-HR',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'UKESM1-0-LL',
 'tif.zip']

# Process 19Band Tiff per model

### Objective:

For each Scenario, for each Period:
1. Create a directory for each model
2. Read and split the 19 band tif
3. Write single band tif to model directory

In [71]:
listdir = ['2021_2040', '2041_2060', '2061_2080', '2081_2100']

models = ['CanESMS_CanOE',
             'CMCC_ESM2',
             'CNRM_CM6_1_HR',
             'CNRM_CM6_1',
             'CNRM_ESM2_1',
             'EC_Earth3_Veg_LR',
             'EC_Earth3_Veg',
             'FIO_ESM_2_0',
             'GISS_E2_1_G',
             'GISS_E2_1_H',
             'HadGEM3_GC31_LL',
             'INM_CM4_8',
             'MIROC6',
             'IPSL_CM6A_LR',
             'INM_CM5_0',
             'MIROC_ES2L',
             'MPI_ESM1_2_HR',
             'MPI_ESM1_2_LR',
             'MRI_ESM2_0',
             'ACCESS-CM2',
             'ACCESS-ESM1',
             'BCC-CSM2-MR',
             'CanESM5',
             'UKESM1_0_LL']

### Walk through each Scenario
for i in os.listdir():
    for s in ['SSP126']:
        if i == s:
            Sdir = os.path.join(s)
            print('Scenario:',Sdir)    
### Walk through each Period
            for year in os.listdir(Sdir):
                print('Year:',year)
                Ydir = os.path.join(s,year)
                
                for i in os.listdir(Ydir):
                    if i.endswith('.tif'):
### Define the file and model
                        fileN = os.path.join(Ydir,i)
                        model = i.split('_')[3]                       
### Create directory for Model
                        modDir = os.path.join(Ydir,model)
                        if not os.path.exists(modDir):
                            print(modDir)
                            os.mkdir(modDir)                    
### Read the 19 Band Tif for the model
                        src = rio.open(fileN)
    
                        for band in range(1, src.count + 1):
                            single_band = src.read(band)
                        
                            # get the output name
                            out_name = os.path.basename(fileN)
                            file, ext = os.path.splitext(out_name)
                            name = "B" + str(band) + ".tif"
                        
                            out_img = os.path.join(modDir, name)
                            print(out_img + " done")
                            
                            # Copy the metadata
                            out_meta = src.meta.copy()
                            out_meta.update({"count": 1})
        
## Save each band to file 
                            with rio.open(out_img, "w", **out_meta) as dest:
                                dest.write(single_band, 1)            

Scenario: SSP126
Year: 2021_2040
SSP126/2021_2040/GFDL-ESM4
SSP126/2021_2040/GFDL-ESM4/B1.tif done
SSP126/2021_2040/GFDL-ESM4/B2.tif done
SSP126/2021_2040/GFDL-ESM4/B3.tif done
SSP126/2021_2040/GFDL-ESM4/B4.tif done
SSP126/2021_2040/GFDL-ESM4/B5.tif done
SSP126/2021_2040/GFDL-ESM4/B6.tif done
SSP126/2021_2040/GFDL-ESM4/B7.tif done
SSP126/2021_2040/GFDL-ESM4/B8.tif done
SSP126/2021_2040/GFDL-ESM4/B9.tif done
SSP126/2021_2040/GFDL-ESM4/B10.tif done
SSP126/2021_2040/GFDL-ESM4/B11.tif done
SSP126/2021_2040/GFDL-ESM4/B12.tif done
SSP126/2021_2040/GFDL-ESM4/B13.tif done
SSP126/2021_2040/GFDL-ESM4/B14.tif done
SSP126/2021_2040/GFDL-ESM4/B15.tif done
SSP126/2021_2040/GFDL-ESM4/B16.tif done
SSP126/2021_2040/GFDL-ESM4/B17.tif done
SSP126/2021_2040/GFDL-ESM4/B18.tif done
SSP126/2021_2040/GFDL-ESM4/B19.tif done
SSP126/2021_2040/UKESM1-0-LL
SSP126/2021_2040/UKESM1-0-LL/B1.tif done
SSP126/2021_2040/UKESM1-0-LL/B2.tif done
SSP126/2021_2040/UKESM1-0-LL/B3.tif done
SSP126/2021_2040/UKESM1-0-LL/B4.tif d

SSP126/2021_2040/CNRM-ESM2-1/B5.tif done
SSP126/2021_2040/CNRM-ESM2-1/B6.tif done
SSP126/2021_2040/CNRM-ESM2-1/B7.tif done
SSP126/2021_2040/CNRM-ESM2-1/B8.tif done
SSP126/2021_2040/CNRM-ESM2-1/B9.tif done
SSP126/2021_2040/CNRM-ESM2-1/B10.tif done
SSP126/2021_2040/CNRM-ESM2-1/B11.tif done
SSP126/2021_2040/CNRM-ESM2-1/B12.tif done
SSP126/2021_2040/CNRM-ESM2-1/B13.tif done
SSP126/2021_2040/CNRM-ESM2-1/B14.tif done
SSP126/2021_2040/CNRM-ESM2-1/B15.tif done
SSP126/2021_2040/CNRM-ESM2-1/B16.tif done
SSP126/2021_2040/CNRM-ESM2-1/B17.tif done
SSP126/2021_2040/CNRM-ESM2-1/B18.tif done
SSP126/2021_2040/CNRM-ESM2-1/B19.tif done
SSP126/2021_2040/EC-Earth3-Veg
SSP126/2021_2040/EC-Earth3-Veg/B1.tif done
SSP126/2021_2040/EC-Earth3-Veg/B2.tif done
SSP126/2021_2040/EC-Earth3-Veg/B3.tif done
SSP126/2021_2040/EC-Earth3-Veg/B4.tif done
SSP126/2021_2040/EC-Earth3-Veg/B5.tif done
SSP126/2021_2040/EC-Earth3-Veg/B6.tif done
SSP126/2021_2040/EC-Earth3-Veg/B7.tif done
SSP126/2021_2040/EC-Earth3-Veg/B8.tif done


SSP126/2021_2040/MIROC-ES2L/B4.tif done
SSP126/2021_2040/MIROC-ES2L/B5.tif done
SSP126/2021_2040/MIROC-ES2L/B6.tif done
SSP126/2021_2040/MIROC-ES2L/B7.tif done
SSP126/2021_2040/MIROC-ES2L/B8.tif done
SSP126/2021_2040/MIROC-ES2L/B9.tif done
SSP126/2021_2040/MIROC-ES2L/B10.tif done
SSP126/2021_2040/MIROC-ES2L/B11.tif done
SSP126/2021_2040/MIROC-ES2L/B12.tif done
SSP126/2021_2040/MIROC-ES2L/B13.tif done
SSP126/2021_2040/MIROC-ES2L/B14.tif done
SSP126/2021_2040/MIROC-ES2L/B15.tif done
SSP126/2021_2040/MIROC-ES2L/B16.tif done
SSP126/2021_2040/MIROC-ES2L/B17.tif done
SSP126/2021_2040/MIROC-ES2L/B18.tif done
SSP126/2021_2040/MIROC-ES2L/B19.tif done
SSP126/2021_2040/MIROC6
SSP126/2021_2040/MIROC6/B1.tif done
SSP126/2021_2040/MIROC6/B2.tif done
SSP126/2021_2040/MIROC6/B3.tif done
SSP126/2021_2040/MIROC6/B4.tif done
SSP126/2021_2040/MIROC6/B5.tif done
SSP126/2021_2040/MIROC6/B6.tif done
SSP126/2021_2040/MIROC6/B7.tif done
SSP126/2021_2040/MIROC6/B8.tif done
SSP126/2021_2040/MIROC6/B9.tif done
SS

SSP126/2041_2060/CanESM5/B8.tif done
SSP126/2041_2060/CanESM5/B9.tif done
SSP126/2041_2060/CanESM5/B10.tif done
SSP126/2041_2060/CanESM5/B11.tif done
SSP126/2041_2060/CanESM5/B12.tif done
SSP126/2041_2060/CanESM5/B13.tif done
SSP126/2041_2060/CanESM5/B14.tif done
SSP126/2041_2060/CanESM5/B15.tif done
SSP126/2041_2060/CanESM5/B16.tif done
SSP126/2041_2060/CanESM5/B17.tif done
SSP126/2041_2060/CanESM5/B18.tif done
SSP126/2041_2060/CanESM5/B19.tif done
SSP126/2041_2060/CanESM5-CanOE
SSP126/2041_2060/CanESM5-CanOE/B1.tif done
SSP126/2041_2060/CanESM5-CanOE/B2.tif done
SSP126/2041_2060/CanESM5-CanOE/B3.tif done
SSP126/2041_2060/CanESM5-CanOE/B4.tif done
SSP126/2041_2060/CanESM5-CanOE/B5.tif done
SSP126/2041_2060/CanESM5-CanOE/B6.tif done
SSP126/2041_2060/CanESM5-CanOE/B7.tif done
SSP126/2041_2060/CanESM5-CanOE/B8.tif done
SSP126/2041_2060/CanESM5-CanOE/B9.tif done
SSP126/2041_2060/CanESM5-CanOE/B10.tif done
SSP126/2041_2060/CanESM5-CanOE/B11.tif done
SSP126/2041_2060/CanESM5-CanOE/B12.tif d

SSP126/2041_2060/GISS-E2-1-H/B6.tif done
SSP126/2041_2060/GISS-E2-1-H/B7.tif done
SSP126/2041_2060/GISS-E2-1-H/B8.tif done
SSP126/2041_2060/GISS-E2-1-H/B9.tif done
SSP126/2041_2060/GISS-E2-1-H/B10.tif done
SSP126/2041_2060/GISS-E2-1-H/B11.tif done
SSP126/2041_2060/GISS-E2-1-H/B12.tif done
SSP126/2041_2060/GISS-E2-1-H/B13.tif done
SSP126/2041_2060/GISS-E2-1-H/B14.tif done
SSP126/2041_2060/GISS-E2-1-H/B15.tif done
SSP126/2041_2060/GISS-E2-1-H/B16.tif done
SSP126/2041_2060/GISS-E2-1-H/B17.tif done
SSP126/2041_2060/GISS-E2-1-H/B18.tif done
SSP126/2041_2060/GISS-E2-1-H/B19.tif done
SSP126/2041_2060/HadGEM3-GC31-LL
SSP126/2041_2060/HadGEM3-GC31-LL/B1.tif done
SSP126/2041_2060/HadGEM3-GC31-LL/B2.tif done
SSP126/2041_2060/HadGEM3-GC31-LL/B3.tif done
SSP126/2041_2060/HadGEM3-GC31-LL/B4.tif done
SSP126/2041_2060/HadGEM3-GC31-LL/B5.tif done
SSP126/2041_2060/HadGEM3-GC31-LL/B6.tif done
SSP126/2041_2060/HadGEM3-GC31-LL/B7.tif done
SSP126/2041_2060/HadGEM3-GC31-LL/B8.tif done
SSP126/2041_2060/HadGEM

SSP126/2061_2080/ACCESS-CM2/B8.tif done
SSP126/2061_2080/ACCESS-CM2/B9.tif done
SSP126/2061_2080/ACCESS-CM2/B10.tif done
SSP126/2061_2080/ACCESS-CM2/B11.tif done
SSP126/2061_2080/ACCESS-CM2/B12.tif done
SSP126/2061_2080/ACCESS-CM2/B13.tif done
SSP126/2061_2080/ACCESS-CM2/B14.tif done
SSP126/2061_2080/ACCESS-CM2/B15.tif done
SSP126/2061_2080/ACCESS-CM2/B16.tif done
SSP126/2061_2080/ACCESS-CM2/B17.tif done
SSP126/2061_2080/ACCESS-CM2/B18.tif done
SSP126/2061_2080/ACCESS-CM2/B19.tif done
SSP126/2061_2080/ACCESS-ESM1-5
SSP126/2061_2080/ACCESS-ESM1-5/B1.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B2.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B3.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B4.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B5.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B6.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B7.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B8.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B9.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B10.tif done
SSP126/2061_2080/ACCESS-ESM1-5/B11.tif done
SSP1

SSP126/2061_2080/EC-Earth3-Veg-LR/B8.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B9.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B10.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B11.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B12.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B13.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B14.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B15.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B16.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B17.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B18.tif done
SSP126/2061_2080/EC-Earth3-Veg-LR/B19.tif done
SSP126/2061_2080/FIO-ESM-2-0
SSP126/2061_2080/FIO-ESM-2-0/B1.tif done
SSP126/2061_2080/FIO-ESM-2-0/B2.tif done
SSP126/2061_2080/FIO-ESM-2-0/B3.tif done
SSP126/2061_2080/FIO-ESM-2-0/B4.tif done
SSP126/2061_2080/FIO-ESM-2-0/B5.tif done
SSP126/2061_2080/FIO-ESM-2-0/B6.tif done
SSP126/2061_2080/FIO-ESM-2-0/B7.tif done
SSP126/2061_2080/FIO-ESM-2-0/B8.tif done
SSP126/2061_2080/FIO-ESM-2-0/B9.tif done
SSP126/2061_2080/FIO-ESM-2-0/B10.tif don

SSP126/2061_2080/MPI-ESM1-2-HR/B12.tif done
SSP126/2061_2080/MPI-ESM1-2-HR/B13.tif done
SSP126/2061_2080/MPI-ESM1-2-HR/B14.tif done
SSP126/2061_2080/MPI-ESM1-2-HR/B15.tif done
SSP126/2061_2080/MPI-ESM1-2-HR/B16.tif done
SSP126/2061_2080/MPI-ESM1-2-HR/B17.tif done
SSP126/2061_2080/MPI-ESM1-2-HR/B18.tif done
SSP126/2061_2080/MPI-ESM1-2-HR/B19.tif done
SSP126/2061_2080/MPI-ESM1-2-LR
SSP126/2061_2080/MPI-ESM1-2-LR/B1.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B2.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B3.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B4.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B5.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B6.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B7.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B8.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B9.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B10.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B11.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B12.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B13.tif done
SSP126/2061_2080/MPI-ESM1-2-LR/B14.tif done
SSP126/206

SSP126/2081_2100/CanESM5/B17.tif done
SSP126/2081_2100/CanESM5/B18.tif done
SSP126/2081_2100/CanESM5/B19.tif done
SSP126/2081_2100/CanESM5-CanOE
SSP126/2081_2100/CanESM5-CanOE/B1.tif done
SSP126/2081_2100/CanESM5-CanOE/B2.tif done
SSP126/2081_2100/CanESM5-CanOE/B3.tif done
SSP126/2081_2100/CanESM5-CanOE/B4.tif done
SSP126/2081_2100/CanESM5-CanOE/B5.tif done
SSP126/2081_2100/CanESM5-CanOE/B6.tif done
SSP126/2081_2100/CanESM5-CanOE/B7.tif done
SSP126/2081_2100/CanESM5-CanOE/B8.tif done
SSP126/2081_2100/CanESM5-CanOE/B9.tif done
SSP126/2081_2100/CanESM5-CanOE/B10.tif done
SSP126/2081_2100/CanESM5-CanOE/B11.tif done
SSP126/2081_2100/CanESM5-CanOE/B12.tif done
SSP126/2081_2100/CanESM5-CanOE/B13.tif done
SSP126/2081_2100/CanESM5-CanOE/B14.tif done
SSP126/2081_2100/CanESM5-CanOE/B15.tif done
SSP126/2081_2100/CanESM5-CanOE/B16.tif done
SSP126/2081_2100/CanESM5-CanOE/B17.tif done
SSP126/2081_2100/CanESM5-CanOE/B18.tif done
SSP126/2081_2100/CanESM5-CanOE/B19.tif done
SSP126/2081_2100/CMCC-ESM2
S

SSP126/2081_2100/GISS-E2-1-H/B16.tif done
SSP126/2081_2100/GISS-E2-1-H/B17.tif done
SSP126/2081_2100/GISS-E2-1-H/B18.tif done
SSP126/2081_2100/GISS-E2-1-H/B19.tif done
SSP126/2081_2100/HadGEM3-GC31-LL
SSP126/2081_2100/HadGEM3-GC31-LL/B1.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B2.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B3.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B4.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B5.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B6.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B7.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B8.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B9.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B10.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B11.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B12.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B13.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B14.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B15.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B16.tif done
SSP126/2081_2100/HadGEM3-GC31-LL/B17.tif done
SSP126/2081_2100/HadGEM3-G

# Convert single band tif to ASCII

### Objective

For each mode, for each Period:
1. Create a directory for ASCII files
2. Convert to ASCII
3. Rename 

In [72]:
for i in os.listdir():
### Walk through each Scenario
    for s in ['SSP126']:
        if i == s:
            Sdir = os.path.join(s)
            print('Scenario:',Sdir)
### Walk through each Period            
            for year in os.listdir(Sdir):
                Ydir = os.path.join(s,year)
                for i in os.listdir(Ydir):
                    if i.endswith('.tif'):
### Define the file and model
                        fileN = os.path.join(Ydir,i)
                        model = i.split('_')[3]
                        modDir = os.path.join(Ydir,model)
### Create directory for ASCII
                        asciiDir = os.path.join(modDir,'ASCII')
                        if not os.path.exists(asciiDir):
                            os.mkdir(asciiDir)
### Write to ASCII
                        for i in os.listdir(modDir):
                            print("Doing:",i)
                            inputfile = i
                            outputfile = inputfile[:-4] + ".asc"
                            command = "gdal_translate  -of AAIGrid -ot Int32 %s %s \n" % (os.path.join(modDir,inputfile), os.path.join(asciiDir,outputfile))
                            proc = subprocess.Popen(command, shell=True)
                            print("Done:",proc.wait())

Scenario: SSP126
Doing: B1.tif
Input file size is 829, 1222
0...10...20...30...40...50...

Warning 1: for band 1, nodata value has been rounded to -2147483648, Int32 being an integer datatype.


60...70...80...90...100 - done.
Done: 0
Doing: B2.tif
Input file size is 829, 1222
0...10

Warning 1: for band 1, nodata value has been rounded to -2147483648, Int32 being an integer datatype.


...20...30...40...50...60...70...80...90...100 - done.
Done: 0
Doing: B3.tif


...

Warning 1: for band 1, nodata value has been rounded to -2147483648, Int32 being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to -2147483648, Int32 being an integer datatype.


.70...80...90...100 - done.
Done: 0
Doing: B16.tif
Input file size is 829, 1222
0...10...20...30...40...50...60...70...80...90...100 - done.
Done: 0
Doing: B17.tif
Input file size is 829, 1222
0...10...20...30...40...50...60...70...80...90...

Warning 1: for band 1, nodata value has been rounded to -2147483648, Int32 being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to -2147483648, Int32 being an integer datatype.


100 - done.
Done: 0
Doing: B18.tif
Input file size is 829, 1222
0...10...20...30...40...50...60...70...80...90...100 - done.
Done: 0
Doing: B19.tif
Input file size is 829, 1222
0...10...20...30...40...50...60...70...80...90...100 - done.
Done: 0
Doing: ASCII
Done: 1


Warning 1: for band 1, nodata value has been rounded to -2147483648, Int32 being an integer datatype.
ERROR 4: `SSP126/2081_2100/MRI-ESM2-0/ASCII' not recognized as a supported file format.


# Run MAXENT

### Objective:

For each Scenario, for each period, for each model:
1. Run the maxent model
2. Write outputs to Output folder

In [4]:
models = [ 'EC_Earth3_Veg_LR',
             'EC_Earth3_Veg',
             'FIO_ESM_2_0',
             'GISS_E2_1_G',
             'GISS_E2_1_H',
             'HadGEM3_GC31_LL',
             'INM_CM4_8',
             'MIROC6',
             'IPSL_CM6A_LR',
             'INM_CM5_0',
             'MIROC_ES2L',
             'MPI_ESM1_2_HR',
             'MPI_ESM1_2_LR',
             'MRI_ESM2_0',
             'ACCESS-CM2',
             'ACCESS-ESM1',
             'BCC-CSM2-MR',
             'CanESM5',
             'UKESM1_0_LL']


for i in os.listdir():
### Walk through each Scenario
    for s in ['SSP585']:
        if i == s:
            print('Scenario:',s)
            Sdir = os.path.join(s)
### Walk through each Period 
            for year in os.listdir(Sdir):
                if year == '2041_2060':
                    print('Year:',year)
                    Ydir = os.path.join(s,year)
                    
                    for model in os.listdir(Ydir):
                        if os.path.isdir(os.path.join(Ydir,model)):
                            if model in models: 
                                now = datetime.now()
                                current_time = now.strftime("%H:%M:%S")
                                print("Running Model:",model, current_time)
                                Mdir = os.path.join(Ydir,model+'/ASCII')
### Create directory for ASCII
                                OutputDir = os.path.join(Ydir,model+'/Output')
                                if not os.path.exists(OutputDir):
                                    os.mkdir(OutputDir)
### Write MAXENT to Output
                                #maxent_cmd = "java -mx1024m -jar /home/Azure_linux_admin/Projdir/Mauritius_IAS/maxent/maxent.jar visible=false environmentallayers=/home/Azure_linux_admin/Projdir/Mauritius_IAS/Current/ASCII samplesfile=/home/Azure_linux_admin/Projdir/Mauritius_IAS/Species/species.csv projectionlayers=/home/Azure_linux_admin/Projdir/Mauritius_IAS/" +Mdir +" " +"outputdirectory=/home/Azure_linux_admin/Projdir/Mauritius_IAS/" +OutputDir+" " +"randomseed=true randomtestpoints=30 replicates=10 replicatetype=Subsample maximumiterations=5000 warnings=false -r redoifexists -a"
                                #proc = subprocess.Popen(maxent_cmd, shell=True)
                                print("Done with:", model,proc.wait())

Scenario: SSP585
Year: 2041_2060
Running Model: ACCESS-CM2 06:11:57


NameError: name 'proc' is not defined

# Get Median for each Specie

## Check that all Models have correct MAXENT Output : 135168

### SSP585 : 24 Models

- Should have 24*4 = 96 Output Folders

In [15]:
### Count folders
%ls -l -d SSP585/*/*/Output | wc -l

96


In [28]:
### Show files
%ls -l -d SSP585/*/*/Output

drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:04 SSP585/2021_2040/ACCESS-CM2/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:10 SSP585/2021_2040/ACCESS-ESM1-5/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:17 SSP585/2021_2040/BCC-CSM2-MR/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:37 SSP585/2021_2040/CMCC-ESM2/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:44 SSP585/2021_2040/CNRM-CM6-1-HR/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:51 SSP585/2021_2040/CNRM-CM6-1/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:58 SSP585/2021_2040/CNRM-ESM2-1/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:24 SSP585/2021_2040/CanESM5-CanOE/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 13 22:31 SSP585/2021_2040/CanESM5/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin

### SSP370 : 23 Models

- Should have 23*4 = 92 Output Folders

In [29]:
### Count folders
%ls -l -d SSP370/*/*/Output | wc -l

92


In [30]:
### Show files
%ls -l -d SSP370/*/*/Output

drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 12:38 SSP370/2021_2040/ACCESS-CM2/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 12:45 SSP370/2021_2040/ACCESS-ESM1-5/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 12:53 SSP370/2021_2040/BCC-CSM2-MR/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 13:00 SSP370/2021_2040/CMCC-ESM2/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 13:07 SSP370/2021_2040/CNRM-CM6-1-HR/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 13:13 SSP370/2021_2040/CNRM-CM6-1/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 13:21 SSP370/2021_2040/CNRM-ESM2-1/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 13:28 SSP370/2021_2040/CanESM5-CanOE/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 14 13:35 SSP370/2021_2040/CanESM5/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin

### SSP245: 24 Models

- Should have 24*4 = 96 Output Folders

In [31]:
### Count folders
%ls -l -d SSP245/*/*/Output | wc -l

96


In [32]:
### Show files
%ls -l -d SSP245/*/*/Output

drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 09:09 SSP245/2021_2040/ACCESS-CM2/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 09:15 SSP245/2021_2040/ACCESS-ESM1-5/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 09:22 SSP245/2021_2040/BCC-CSM2-MR/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 09:29 SSP245/2021_2040/CMCC-ESM2/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 09:36 SSP245/2021_2040/CNRM-CM6-1-HR/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 09:43 SSP245/2021_2040/CNRM-CM6-1/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 09:50 SSP245/2021_2040/CNRM-ESM2-1/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 09:57 SSP245/2021_2040/CanESM5-CanOE/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 18 10:04 SSP245/2021_2040/CanESM5/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin

### SSP 126: 25 Models

- Should have 25 * 4 = 100

In [33]:
### Count folders
%ls -l -d SSP126/*/*/Output | wc -l

100


In [34]:
### Show files
%ls -l -d SSP126/*/*/Output

drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:33 SSP126/2021_2040/ACCESS-CM2/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:33 SSP126/2021_2040/ACCESS-ESM1-5/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:33 SSP126/2021_2040/BCC-CSM2-MR/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:35 SSP126/2021_2040/CMCC-ESM2/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:36 SSP126/2021_2040/CNRM-CM6-1-HR/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:35 SSP126/2021_2040/CNRM-CM6-1/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:36 SSP126/2021_2040/CNRM-ESM2-1/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:34 SSP126/2021_2040/CanESM5-CanOE/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin 135168 Jul 19 09:34 SSP126/2021_2040/CanESM5/Output/
drwxrwxr-x 3 Azure_linux_admin Azure_linux_admin

## Write Model Median for each specie

In [4]:
species = ['Acacia_nilotica']

for i in os.listdir():
### Walk through each Scenario
    for s in ['SSP245']:
        if i == s:
            print('Scenario:',s)
            Sdir = os.path.join(s)
### Walk through each Period 
            for year in os.listdir(Sdir):
                print('Year:',year)
                Ydir = os.path.join(s,year)
                SpecieDir = os.path.join('Species_Output')
                if not os.path.exists(SpecieDir):
                    os.mkdir(SpecieDir)
                for specie in species:
                    spec = specie + '_ASCII_avg.asc'
                    print(specie)
                    filelist = files = glob.glob(os.path.join(r'{}'.format(Ydir),'*','Output','{}'.format(spec)))
                    tci_images = []
                    profile = None
                    for file in filelist:
                        if profile is None:
                            temp_img = rio.open(file)
                            profile = temp_img.profile
                        tci_images.append(rxr.open_rasterio(file, chunks={'x': 1000, 'y': 1000}))
                    comp = xr.concat(tci_images, dim='band')
                    print('Getting Median for:', specie)
                    median = comp.median(dim='band', keep_attrs=True,skipna=True)
                    print('Writing Median to:',SpecieDir)
                    median.rio.to_raster(os.path.join(SpecieDir, specie+'_{}_'.format(s)+'_{}_'.format(year)+'ASCII.tif'))

Scenario: SSP245
Year: 2021_2040
Acacia_nilotica
Getting Median for: Acacia_nilotica
Writing Median to: Species_Output
Year: 2041_2060
Acacia_nilotica
Getting Median for: Acacia_nilotica
Writing Median to: Species_Output
Year: 2061_2080
Acacia_nilotica
Getting Median for: Acacia_nilotica
Writing Median to: Species_Output
Year: 2081_2100
Acacia_nilotica
Getting Median for: Acacia_nilotica
Writing Median to: Species_Output


In [20]:
species = ['Acacia_nilotica','Albizia_procera','Clibadium_surinamense_L.',
           'Hedychium_flavescens_Carey_ex_Roscoe',
           'Hiptage_benghalensis__L.__Kurz','Leucaena_leucocephala__Lam.__De_Wit',
           'Pithecellobium_dulce__Roxb.__Benth.','Pithecellobium_unguis-cati__L.__Benth.','Psidium_cattleianum_Afzel._ex_Sabine',
           'Ravenala_madagascariensis_Sonn.','Schinus_terebinthifolius_Raddi',
           'Syzygium_jambos__L.__Alston','Tabebuia_pallida__Lindl.__Miers',
           'Tecoma_stans__L.__Juss._ex_Kunth']

for i in os.listdir():
### Walk through each Scenario
    for s in ['SSP126','SSP245','SSP370','SSP585']:
        if i == s:
            print('Scenario:',s)
            Sdir = os.path.join(s)
### Walk through each Period 
            for year in os.listdir(Sdir):
                print('Year:',year)
                Ydir = os.path.join(s,year)
                SpecieDir = os.path.join('Species_Output_New')
                if not os.path.exists(SpecieDir):
                    os.mkdir(SpecieDir)
                for specie in species:
                    spec = specie + '_ASCII_avg.asc'
                    print(specie)
                    filelist = files = glob.glob(os.path.join(r'{}'.format(Ydir),'*','Output','{}'.format(spec)))
                    tci_images = []
                    profile = None
                    for file in filelist:
                        if profile is None:
                            temp_img = rio.open(file)
                            profile = temp_img.profile
                        tci_images.append(rxr.open_rasterio(file, chunks={'x': 1000, 'y': 1000}))
                    comp = xr.concat(tci_images, dim='band')
                    print('Getting Median for:', specie)
                    median = comp.median(dim='band', keep_attrs=True,skipna=True)
                    print('Writing Median to:',SpecieDir)
                    median.rio.to_raster(os.path.join(SpecieDir, specie+'_{}_'.format(s)+'_{}_'.format(year)+'.tif'))

Scenario: SSP585
Year: 2021_2040
Acacia_nilotica
Getting Median for: Acacia_nilotica
Writing Median to: Species_Output_New
Albizia_procera
Getting Median for: Albizia_procera
Writing Median to: Species_Output_New
Clibadium_surinamense_L.
Getting Median for: Clibadium_surinamense_L.
Writing Median to: Species_Output_New
Hedychium_flavescens_Carey_ex_Roscoe
Getting Median for: Hedychium_flavescens_Carey_ex_Roscoe
Writing Median to: Species_Output_New
Hiptage_benghalensis__L.__Kurz
Getting Median for: Hiptage_benghalensis__L.__Kurz
Writing Median to: Species_Output_New
Leucaena_leucocephala__Lam.__De_Wit
Getting Median for: Leucaena_leucocephala__Lam.__De_Wit
Writing Median to: Species_Output_New
Pithecellobium_dulce__Roxb.__Benth.
Getting Median for: Pithecellobium_dulce__Roxb.__Benth.
Writing Median to: Species_Output_New
Pithecellobium_unguis-cati__L.__Benth.
Getting Median for: Pithecellobium_unguis-cati__L.__Benth.
Writing Median to: Species_Output_New
Psidium_cattleianum_Afzel._ex_S

Tabebuia_pallida__Lindl.__Miers
Getting Median for: Tabebuia_pallida__Lindl.__Miers
Writing Median to: Species_Output_New
Tecoma_stans__L.__Juss._ex_Kunth
Getting Median for: Tecoma_stans__L.__Juss._ex_Kunth
Writing Median to: Species_Output_New
Year: 2041_2060
Acacia_nilotica
Getting Median for: Acacia_nilotica
Writing Median to: Species_Output_New
Albizia_procera
Getting Median for: Albizia_procera
Writing Median to: Species_Output_New
Clibadium_surinamense_L.
Getting Median for: Clibadium_surinamense_L.
Writing Median to: Species_Output_New
Hedychium_flavescens_Carey_ex_Roscoe
Getting Median for: Hedychium_flavescens_Carey_ex_Roscoe
Writing Median to: Species_Output_New
Hiptage_benghalensis__L.__Kurz
Getting Median for: Hiptage_benghalensis__L.__Kurz
Writing Median to: Species_Output_New
Leucaena_leucocephala__Lam.__De_Wit
Getting Median for: Leucaena_leucocephala__Lam.__De_Wit
Writing Median to: Species_Output_New
Pithecellobium_dulce__Roxb.__Benth.
Getting Median for: Pithecellobi

Schinus_terebinthifolius_Raddi
Getting Median for: Schinus_terebinthifolius_Raddi
Writing Median to: Species_Output_New
Syzygium_jambos__L.__Alston
Getting Median for: Syzygium_jambos__L.__Alston
Writing Median to: Species_Output_New
Tabebuia_pallida__Lindl.__Miers
Getting Median for: Tabebuia_pallida__Lindl.__Miers
Writing Median to: Species_Output_New
Tecoma_stans__L.__Juss._ex_Kunth
Getting Median for: Tecoma_stans__L.__Juss._ex_Kunth
Writing Median to: Species_Output_New
Year: 2061_2080
Acacia_nilotica
Getting Median for: Acacia_nilotica
Writing Median to: Species_Output_New
Albizia_procera
Getting Median for: Albizia_procera
Writing Median to: Species_Output_New
Clibadium_surinamense_L.
Getting Median for: Clibadium_surinamense_L.
Writing Median to: Species_Output_New
Hedychium_flavescens_Carey_ex_Roscoe
Getting Median for: Hedychium_flavescens_Carey_ex_Roscoe
Writing Median to: Species_Output_New
Hiptage_benghalensis__L.__Kurz
Getting Median for: Hiptage_benghalensis__L.__Kurz
Wr

Psidium_cattleianum_Afzel._ex_Sabine
Getting Median for: Psidium_cattleianum_Afzel._ex_Sabine
Writing Median to: Species_Output_New
Ravenala_madagascariensis_Sonn.
Getting Median for: Ravenala_madagascariensis_Sonn.
Writing Median to: Species_Output_New
Schinus_terebinthifolius_Raddi
Getting Median for: Schinus_terebinthifolius_Raddi
Writing Median to: Species_Output_New
Syzygium_jambos__L.__Alston
Getting Median for: Syzygium_jambos__L.__Alston
Writing Median to: Species_Output_New
Tabebuia_pallida__Lindl.__Miers
Getting Median for: Tabebuia_pallida__Lindl.__Miers
Writing Median to: Species_Output_New
Tecoma_stans__L.__Juss._ex_Kunth
Getting Median for: Tecoma_stans__L.__Juss._ex_Kunth
Writing Median to: Species_Output_New
Year: 2041_2060
Acacia_nilotica
Getting Median for: Acacia_nilotica
Writing Median to: Species_Output_New
Albizia_procera
Getting Median for: Albizia_procera
Writing Median to: Species_Output_New
Clibadium_surinamense_L.
Getting Median for: Clibadium_surinamense_L.
